<a href="https://colab.research.google.com/github/buganart/simple-pytorch-3dgan/blob/master/train_simple3D_dcgan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Setup
# @markdown 1. Before starting please save the notebook in your drive by clicking on `File -> Save a copy in drive`
# @markdown 2. Check GPU
# @markdown 3. Mount google drive.
# @markdown 4. Log in to wandb.


!nvidia-smi -L
import os

print(f"We have {os.cpu_count()} CPU cores.")
print()

try:
    from google.colab import drive, output

    IN_COLAB = True
except ImportError:
    from IPython.display import clear_output

    IN_COLAB = False

from pathlib import Path

if IN_COLAB:
    drive.mount("/content/drive/")

    if not Path("/content/drive/My Drive/IRCMS_GAN_collaborative_database").exists():
        raise RuntimeError(
            "Shortcut to our shared drive folder doesn't exits.\n\n"
            "\t1. Go to the google drive web UI\n"
            '\t2. Right click shared folder IRCMS_GAN_collaborative_database and click "Add shortcut to Drive"'
        )

clear = output.clear if IN_COLAB else clear_output


def clear_on_success(msg="Ok!"):
    if _exit_code == 0:
        clear()
        print(msg)


print()
print("Wandb installation and login ...")
%pip install -q wandb

wandb_drive_netrc_path = Path("drive/My Drive/colab/.netrc")
wandb_local_netrc_path = Path("/root/.netrc")
if wandb_drive_netrc_path.exists():
    import shutil

    print("Wandb .netrc file found, will use that to log in.")
    shutil.copy(wandb_drive_netrc_path, wandb_local_netrc_path)
else:
    print(
        f"Wandb config not found at {wandb_drive_netrc_path}.\n"
        f"Using manual login.\n\n"
        f"To use auto login in the future, finish the manual login first and then run:\n\n"
        f"\t!mkdir -p '{wandb_drive_netrc_path.parent}'\n"
        f"\t!cp {wandb_local_netrc_path} '{wandb_drive_netrc_path}'\n\n"
        f"Then that file will be used to login next time.\n"
    )

!wandb login

In [ ]:
#@title Configuration

# Fill in the configuration then Then, select `Runtime` and `Run all` then let it ride!

#@markdown #### Training
drive_dirs = list(Path('/content/drive').glob('My*'))
if not drive_dirs:
    raise RuntimeError("Drive not found. Is it mounted?")
drive = drive_dirs[0]
print(f"Google drive at {drive}")    


#@markdown Enter dataset location.  
#@markdown - For example via the file browser on the left to locate and right click to copy the path.)
# #@markdown - zipfile example: `/content/drive/My Drive/h/k/a.zip`
#@markdown - file folder example: `/content/drive/My Drive/h/k`
#@markdown - Currently, only support folder (zip not supported)
# #@markdown - if data_location_option is not empty, data_location_option will overwrite data_location_default
# data_location_default = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Research/Peter/Tree_3D_models_obj_auto_generated/sessions/simplified/tree-session-2020-09-14_23-23-Friedrich_2-target-face-num-1000.zip" #@param ["/content/drive/My Drive/IRCMS_GAN_collaborative_database/Research/Peter/Tree_3D_models_obj_auto_generated/sessions/simplified/tree-session-2020-09-14_23-23-Friedrich_2-target-face-num-1000.zip", "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Research/Peter/Tree_3D_models_obj_auto_generated/sessions/simplified/tree-sessions-2020-09-10-simplified-26k-target-face-num-1000.zip", "/content/drive/My Drive/Hand-Tool-Data-Set/turbosquid_thingiverse_dataset/dataset_ply_out_zipped.zip", "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Research/Peter/Chairs_Princeton/chair_train.zip", "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Research/Peter/Tree_3D_models_obj_auto_generated/sessions/simplified/tree-sessions-2020-09-10-simplified-26k-target-face-num-1000-class-label.zip"] 
data_location = "/content/drive/My Drive/Hand-Tool-Data-Set/Finn/bell_obj" #@param {type:"string"}

if not Path(data_location).exists():
    raise RuntimeError(f"The data_location \'{data_location}\' does not exist.")

# #@markdown [Optional] ID of wandb run to resume.
# resume_run_id = "" #@param {type: "string"}

#@markdown ####WANDB log
#@markdown - how many epochs before logging images/3D objects
log_interval =  10 #@param {type: "integer"}

#@markdown ###train argument
epochs = 1000 #@param {type: "integer"}
#@markdown - !Warning! rotate is very slow in colab.
rotate = False #@param {type:"boolean"}
#@markdown - logs by tensorboardX
logs = "" #@param {type: "string"}
#@markdown - local test verbose
local_test = False #@param {type:"boolean"}
#@markdown - model name for saving
model_name = "dcgan" #@param {type: "string"}
#@markdown - call tester.py
test = False #@param {type:"boolean"}


config = dict(
    data_location=data_location,
    log_interval=log_interval,
    epochs=epochs,
    rotate=rotate,
    logs=logs,
    local_test=local_test,
    model_name=model_name,
    test=test,
)
for k,v in config.items():
    print(f"=> {k:30}: {v}")

In [ ]:
#@title Clone simple-pytorch-3dgan repo.
if IN_COLAB:
    !git clone https://github.com/buganart/simple-pytorch-3dgan
    clear_on_success("Repo cloned!")

In [ ]:
#@title Install dependencies
if IN_COLAB:
    %cd "/content/simple-pytorch-3dgan/"
    
%pip install -r requirements.txt
clear_on_success("Dependencies installed!")

In [ ]:
#@title Install Visdom
! npm install -g localtunnel
get_ipython().system_raw('python3 -m pip install visdom')
get_ipython().system_raw('python3 -m visdom.server -port 6006 >> visdomlog.txt 2>&1 &')
get_ipython().system_raw('lt --port 6006 >> url.txt 2>&1 &')
import time
# time.sleep(5)
import visdom
clear_on_success("visdom installed!")

In [ ]:
#@title Train

# import os
os.environ["WANDB_MODE"] = "dryrun"

#convert parameters to appropriate type
if not logs:
    logs = None
local_test = str(local_test)
test = str(test)

if rotate:
    !env PYTHONPATH="." python main.py \
        --data_dir "$data_location" \
        --generate_every "$log_interval" \
        --epochs "$epochs" \
        --logs "$logs" \
        --local_test "$local_test" \
        --model_name "$model_name" \
        --test "$test" \
        --rotate
else:
    !env PYTHONPATH="." python main.py \
        --data_dir "$data_location" \
        --generate_every "$log_interval" \
        --epochs "$epochs" \
        --logs "$logs" \
        --local_test "$local_test" \
        --model_name "$model_name" \
        --test "$test" \